In [3]:
import numpy as np

np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [4]:
def Quasi_Newton(x_0, H_0, H_update_mechanism):
        
    #Defining the necessary lists...
    x = []
    x_diff = []

    g = []
    g_diff = []

    d = []
    alpha = []
    H = []
    B = []

    x.append(x_0)
    H.append(H_0)
    B.append(H_0)
    
    g.append(grad(x[0]))
    print("g[{0}]:\n".format(0), g[0])
    
    k = 0
    while(True):
        print("\n\nIteration ", k+1)        
        
        if grad_equals_zero(g[k]):    #If none of the elements are true, or, if all of the elements are zero
            return x[k] 
        else:
            d.append(-np.dot(H[k], g[k]))
            print("d[{0}]:\n".format(k), d[k])
        
        # We use the formula for alpha as given by Conjugate direction method,
        # since in the quadratic case the Quasi-Newton algortihm is also a  Conjugate direction algorithm
        alpha.append(-np.dot(g[k].T, d[k]).item()/(np.dot(np.dot(d[k].T, Q), d[k]).item()))
        print("alpha[{0}]: ".format(k), alpha[k])
        
        x_diff.append(alpha[k]*d[k])
        print("x_diff[{0}]:\n".format(k), x_diff[k])
        
        x.append(x[k] + x_diff[k])
        print("x[{0}]:\n".format(k+1), x[k+1])
        
        g.append(grad(x[k+1]))
        print("g[{0}]:\n".format(k+1), g[k+1])
        
        if grad_equals_zero(g[-1]):
            return x[-1]
                
        g_diff.append(g[k+1] - g[k])
        print("g_diff[{0}]:\n".format(k), g_diff[k])
                
        if H_update_mechanism is "BFGS":
            H.append(H_BFGS(x_diff, g_diff, H, k))
            print("H[{0}]:\n".format(k+1), H[k+1])
        
            B.append(B_update(x_diff, g_diff, B, k))
            print("B[{0}]:\n".format(k+1), B[k+1])
        
        elif H_update_mechanism is "DFP":
            H.append(H_DFP(x_diff, g_diff, H, k))
            print("H[{0}]:\n".format(k+1), H[k+1])
        
        elif H_update_mechanism is "R1":
            H.append(H_rankone(x_diff, g_diff, H, k))
            print("H[{0}]:\n".format(k+1), H[k+1])
        
        k = k+1


In [5]:
# Computing H[k+1] given H[k]

def H_BFGS(x_diff, g_diff, H, k):
    
    t1 = np.dot(x_diff[k], x_diff[k].T)/np.dot(x_diff[k].T, g_diff[k]).item()
    
    coeff_t1 = 1 + np.dot(np.dot(g_diff[k].T, H[k]), g_diff[k]).item()/np.dot(g_diff[k].T, x_diff[k]).item()
    
    t1 *= coeff_t1
    
    temp = np.dot(np.dot(H[k], g_diff[k]), x_diff[k].T)
    t2 = (temp + temp.T)/np.dot(g_diff[k].T, x_diff[k]).item()
    
    return H[k] + t1-t2



def H_DFP(x_diff, g_diff, H, k):
        
    t1 = np.dot(x_diff[k], x_diff[k].T)/np.dot(x_diff[k].T, g_diff[k]).item()
    
    temp = np.dot(H[k], g_diff[k])
    t2 = np.dot(temp,temp.T)/np.dot(g_diff[k].T, temp).item()
    
    return H[k] + t1-t2



def H_rankone(x_diff, g_diff, H, k):
    
    t = x_diff[k] - np.dot(H[k], g_diff[k])
    
    Nr = np.dot(t, t.T)
    Dr = np.dot(g_diff[k].T, t).item()
    
    return H[k] + Nr/Dr

In [6]:
def B_update(x_diff, g_diff, B, k):
    
    t1 = np.dot(g_diff[k], g_diff[k].T)/np.dot(g_diff[k].T, x_diff[k]).item()
    
    temp = np.dot(B[k], x_diff[k])
    t2 = np.dot(temp, temp.T)/np.dot(x_diff[k].T, temp).item()
    
    return B[k] + t1-t2

In [7]:
def grad(x):
    return (np.dot(Q, x) - b)

def grad_equals_zero(g):
    return not np.any(np.around(g, decimals=4))

In [8]:
#Implement the algorithm...

#Eg. 11.4
Q = np.array([[5, -3], [-3, 2]])
b = np.array([[0, 1]]).T

# initial values...
x_0 = np.array([[0, 0]]).T
H_0 = np.identity(2)

print("\nThe minimizer: x* =\n", Quasi_Newton(x_0, H_0, "BFGS"))

g[0]:
 [[ 0]
 [-1]]


Iteration  1
d[0]:
 [[-0.]
 [ 1.]]
alpha[0]:  0.5
x_diff[0]:
 [[-0. ]
 [ 0.5]]
x[1]:
 [[0. ]
 [0.5]]
g[1]:
 [[-1.5]
 [ 0. ]]
g_diff[0]:
 [[-1.5]
 [ 1. ]]
H[1]:
 [[1.   1.5 ]
 [1.5  2.75]]
B[1]:
 [[ 5.5 -3. ]
 [-3.   2. ]]


Iteration  2
d[1]:
 [[1.5 ]
 [2.25]]
alpha[1]:  2.0
x_diff[1]:
 [[3. ]
 [4.5]]
x[2]:
 [[3.]
 [5.]]
g[2]:
 [[0.]
 [0.]]

The minimizer: x* =
 [[3.]
 [5.]]


In [9]:
#Implement the algorithm...

Q = np.array([[2, 0], [0, 1]])
b = np.array([[0, 0]]).T

# initial values...
x_0 = np.array([[1, 2]]).T
H_0 = np.identity(2)

print("\nThe minimizer: x* =\n", Quasi_Newton(x_0, H_0, "R1"))

g[0]:
 [[2]
 [2]]


Iteration  1
d[0]:
 [[-2.]
 [-2.]]
alpha[0]:  0.6666666666666666
x_diff[0]:
 [[-1.3333]
 [-1.3333]]
x[1]:
 [[-0.3333]
 [ 0.6667]]
g[1]:
 [[-0.6667]
 [ 0.6667]]
g_diff[0]:
 [[-2.6667]
 [-1.3333]]
H[1]:
 [[0.5 0. ]
 [0.  1. ]]


Iteration  2
d[1]:
 [[ 0.3333]
 [-0.6667]]
alpha[1]:  1.0
x_diff[1]:
 [[ 0.3333]
 [-0.6667]]
x[2]:
 [[0.]
 [0.]]
g[2]:
 [[0.]
 [0.]]

The minimizer: x* =
 [[0.]
 [0.]]


In [17]:
#Implement the algorithm...

Q = np.array([[3, 1], [1, 2]])
b = np.array([[0, 0]]).T

# initial values...
x_0 = np.array([[0, 1]]).T
H_0 = np.identity(2)

print("\nThe minimizer: x* =\n", Quasi_Newton(x_0, H_0, "BFGS"))

g[0]:
 [[1]
 [2]]


Iteration  1
d[0]:
 [[-1.]
 [-2.]]
alpha[0]:  0.3333333333333333
x_diff[0]:
 [[-0.3333]
 [-0.6667]]
x[1]:
 [[-0.3333]
 [ 0.3333]]
g[1]:
 [[-0.6667]
 [ 0.3333]]
g_diff[0]:
 [[-1.6667]
 [-1.6667]]
H[1]:
 [[ 0.6222 -0.4222]
 [-0.4222  0.8222]]
B[1]:
 [[2.4667 1.2667]
 [1.2667 1.8667]]


Iteration  2
d[1]:
 [[ 0.5556]
 [-0.5556]]
alpha[1]:  0.6
x_diff[1]:
 [[ 0.3333]
 [-0.3333]]
x[2]:
 [[0.]
 [0.]]
g[2]:
 [[0.]
 [0.]]

The minimizer: x* =
 [[0.]
 [0.]]
